In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import csv

In [6]:
df = pd.read_csv('cleandataset.csv', header=0)
df.head()

,Unnamed: 0,aid,paper,author,date,url,content,clean_content
0,0,fox0,fox,['Adam Carrington'],2018-03-03 00:00:00,http://www.foxnews.com/opinion/2018/03/03/supr...,The debate about how to deal with guns in our ...,"['debate', 'deal', 'gun', 'country', 'rage', '..."
1,1,fox1,fox,"['Kaitlyn Schallhorn', '- Jonas Oransky', 'Eve...",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/gun...,In the weeks after a gunman walked into a high...,"['week', 'gunman', 'walked', 'high', 'school',..."
2,2,fox2,fox,['Kathleen Joyce'],2018-03-03 00:00:00,http://www.foxnews.com/us/2018/03/03/cmu-stude...,The 19-year-old student suspected of killing h...,"['19-year-old', 'student', 'suspected', 'killi..."
3,4,fox4,fox,['Gregg Re'],2018-03-02 00:00:00,http://www.foxnews.com/politics/2018/03/02/tru...,Just one day after putting the NRA on the defe...,"['one', 'day', 'putting', 'nra', 'defensive', ..."
4,5,fox5,fox,"['Brooke Singman', '- President Trump']",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/tru...,President Trump has touched off new tensions w...,"['president', 'trump', 'ha', 'touched', 'new',..."


In [7]:
#sentiment analyzer
def sentanal(tokens):
    sents=[]
    for tok in tokens:
        blob=TextBlob(tok, analyzer=NaiveBayesAnalyzer())
        sents.append(blob.sentiment)
    return sents

In [8]:
#analyze just the first line of each article
#this could and probably should be much faster!!!
s1=[]
for cont in df['content']:
    myl=cont.split('.')
    line1=myl[0]
    s1.append(line1)

In [9]:
mysent=sentanal(s1)

KeyboardInterrupt: 

In [25]:
#inspect
mysent[:5]

[Sentiment(classification='pos', p_pos=0.9954586638537127, p_neg=0.004541336146284627),
 Sentiment(classification='neg', p_pos=0.23682905598571363, p_neg=0.763170944014283),
 Sentiment(classification='pos', p_pos=0.8995924032002689, p_neg=0.10040759679973492),
 Sentiment(classification='pos', p_pos=0.9989310360900759, p_neg=0.0010689639099280633),
 Sentiment(classification='pos', p_pos=0.9992331286327895, p_neg=0.000766871367211887)]

In [35]:
#make a dataframe out of the results
#is there a better way to do this - more directly? Omit the listing step?
sentlist=[]
poslist=[]
neglist=[]
for item in mysent:
    sentlist.append(item.classification)
    poslist.append(item.p_pos)
    neglist.append(item.p_neg)
df['sent']=sentlist
df['p_pos']=poslist
df['p_neg']=neglist

In [37]:
df.head(5)

,ind,aid,paper,author,date,url,content,clean_content,sent,p_pos,p_neg
0,0,fox0,fox,['Adam Carrington'],2018-03-03 00:00:00,http://www.foxnews.com/opinion/2018/03/03/supr...,The debate about how to deal with guns in our ...,"['debate', 'deal', 'gun', 'country', 'rage', '...",pos,0.995459,0.004541
1,1,fox1,fox,"['Kaitlyn Schallhorn', '- Jonas Oransky', 'Eve...",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/gun...,In the weeks after a gunman walked into a high...,"['week', 'gunman', 'walked', 'high', 'school',...",neg,0.236829,0.763171
2,2,fox2,fox,['Kathleen Joyce'],2018-03-03 00:00:00,http://www.foxnews.com/us/2018/03/03/cmu-stude...,The 19-year-old student suspected of killing h...,"['19-year-old', 'student', 'suspected', 'killi...",pos,0.899592,0.100408
3,4,fox4,fox,['Gregg Re'],2018-03-02 00:00:00,http://www.foxnews.com/politics/2018/03/02/tru...,Just one day after putting the NRA on the defe...,"['one', 'day', 'putting', 'nra', 'defensive', ...",pos,0.998931,0.001069
4,5,fox5,fox,"['Brooke Singman', '- President Trump']",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/tru...,President Trump has touched off new tensions w...,"['president', 'trump', 'ha', 'touched', 'new',...",pos,0.999233,0.000767


In [44]:
#this is too big - drop the content
df=df.drop('content', axis=1)
df=df.drop('clean_content', axis=1)

In [48]:
df.head(5)

,ind,aid,paper,author,date,url,sent,p_pos,p_neg
0,0,fox0,fox,['Adam Carrington'],2018-03-03 00:00:00,http://www.foxnews.com/opinion/2018/03/03/supr...,pos,0.995459,0.004541
1,1,fox1,fox,"['Kaitlyn Schallhorn', '- Jonas Oransky', 'Eve...",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/gun...,neg,0.236829,0.763171
2,2,fox2,fox,['Kathleen Joyce'],2018-03-03 00:00:00,http://www.foxnews.com/us/2018/03/03/cmu-stude...,pos,0.899592,0.100408
3,4,fox4,fox,['Gregg Re'],2018-03-02 00:00:00,http://www.foxnews.com/politics/2018/03/02/tru...,pos,0.998931,0.001069
4,5,fox5,fox,"['Brooke Singman', '- President Trump']",2018-03-01 00:00:00,http://www.foxnews.com/politics/2018/03/01/tru...,pos,0.999233,0.000767


In [49]:
#save the sentiments out
outfile = open('sentiments.csv', 'w')
df.to_csv(outfile)